In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
in_size = 784
hidden_size = 250
num_classes = 10
num_epochs = 5
batch_size = 150
lr = 0.001

In [3]:
train_dataset = dsets.MNIST(root="./datasets/",
                           train=True,
                           transform=transforms.ToTensor(),
                           download=False)

In [4]:
test_dataset = dsets.MNIST(root="./datasets/",
                          train=False,
                          transform=transforms.ToTensor(),
                          download=False)

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [6]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

Define the class for Feed Forward Neural Network.

In [7]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.LeakyReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [8]:
net = Net(input_size=in_size, hidden_size=hidden_size,
          num_classes=num_classes)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [14]:
for i, m in enumerate(net.modules()):
    print(m)

Net (
  (fc1): Linear (784 -> 250)
  (relu): ReLU ()
  (fc2): Linear (250 -> 10)
)
Linear (784 -> 250)
ReLU ()
Linear (250 -> 10)


In [17]:
train_l = iter(train_loader)

In [18]:
blah = train_l.next()

In [25]:
print(blah[0].size())
print(blah[1].size())

torch.Size([150, 1, 28, 28])
torch.Size([150])


In [26]:
blah2 = torch.FloatTensor([1,2])

In [28]:
# Train the model

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Convert torch tensor to Variable
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size,
                     loss.data[0]))

Epoch [1/5], Step [100/400], Loss: 0.3615
Epoch [1/5], Step [200/400], Loss: 0.3481
Epoch [1/5], Step [300/400], Loss: 0.2876
Epoch [1/5], Step [400/400], Loss: 0.1619
Epoch [2/5], Step [100/400], Loss: 0.2164
Epoch [2/5], Step [200/400], Loss: 0.2342
Epoch [2/5], Step [300/400], Loss: 0.0829
Epoch [2/5], Step [400/400], Loss: 0.1586
Epoch [3/5], Step [100/400], Loss: 0.0717
Epoch [3/5], Step [200/400], Loss: 0.0694
Epoch [3/5], Step [300/400], Loss: 0.1169
Epoch [3/5], Step [400/400], Loss: 0.1062
Epoch [4/5], Step [100/400], Loss: 0.0360
Epoch [4/5], Step [200/400], Loss: 0.0823
Epoch [4/5], Step [300/400], Loss: 0.0540
Epoch [4/5], Step [400/400], Loss: 0.0860
Epoch [5/5], Step [100/400], Loss: 0.0491
Epoch [5/5], Step [200/400], Loss: 0.0337
Epoch [5/5], Step [300/400], Loss: 0.1119
Epoch [5/5], Step [400/400], Loss: 0.0565


In [34]:
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%'
      % (100 * correct / total))

Accuracy of the network on the 10000 test images: 97 %


In [ ]:
# save the Model
# torch.save(net.state_dict(), "model.pkl")

In [32]:
blah = torch.randn(4,3)
print(blah)
print(torch.max(blah,1))


-0.2578  0.2810  1.0098
 0.6320 -1.3269  0.2230
 1.0321  0.1511  0.4091
 2.0378  0.6811  0.2187
[torch.FloatTensor of size 4x3]

(
 1.0098
 0.6320
 1.0321
 2.0378
[torch.FloatTensor of size 4x1]
, 
 2
 0
 0
 0
[torch.LongTensor of size 4x1]
)


In [48]:
import numpy as np

if torch.cuda.is_available():
    x = Variable(torch.cuda.FloatTensor(np.random.randn(4,3)),
                requires_grad=True)
    y = Variable(torch.cuda.FloatTensor(np.random.randn(4,3)),
                requires_grad=True)
    z = x*2 + y*3
    print(z)

RuntimeError: backward should be called only on a scalar (i.e. 1-element tensor) or with gradient w.r.t. the variable